Dans ce script, nous allons tester différents modèles de type convolutionnelsur le dataset 

résultat:
* l'utilisation des batchs rend l'inférence globale plus rapide
* un modèle a besoin de beaucoup de temps / parametres pour arriver à un certain niveau d'accuracy

In [ ]:
import retinoto_py as fovea
args = fovea.Params()
# args = fovea.Params(n_train_stop=0, n_val_stop=0)
args

In [ ]:
fovea.all_cn_model_names

# testing different networks

In [ ]:
json_filename = args.data_cache / '14_model_cn_comparison.json'

# %rm {json_filename}  # FORCING RECOMPUTE

if json_filename.exists():
    results = fovea.pd.read_json(json_filename)
else:
    all_results = []
    for do_mask in [True, False]:
        for dataset in fovea.all_datasets:
            args = fovea.Params(do_mask=do_mask)
            VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
            val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, n_stop=args.n_val_stop)
            val_loader = fovea.get_loader(args, val_dataset)
            for model_name in fovea.all_cn_model_names:
                args.do_mask = do_mask
                args.model_name = model_name
                model = fovea.load_model(args)
                param_stats = fovea.count_parameters(model)
                total_layers = fovea.count_layers(model)
                tic = fovea.time.time()
                accuracy = fovea.get_validation_accuracy(args, model, val_loader, f"Model {args.model_name}\t dataset: {dataset}\t(do_mask={do_mask})")            
                toc = fovea.time.time()
                this_result = { 'model_name': model_name,
                                'do_mask': do_mask,
                                'dataset': dataset,
                                'accuracy': accuracy,
                                'wall_clock_time': (toc-tic)/len(val_dataset),
                                'total_parameters': param_stats['total_parameters'],
                                'trainable_parameters': param_stats['trainable_parameters'],
                                'total_layers': total_layers}
                all_results.append(this_result)
    results = fovea.pd.DataFrame(all_results)
    results.to_json(json_filename, orient='records', indent=2)


In [ ]:
results

In [ ]:
fig, axes = fovea.plot_model_comparison(results, fovea.all_cn_model_names, fovea.all_datasets, do_masks=[True, False], figures_folder=args.figures_folder, save_name='14_model_convnext', exts=['svg'])


In [ ]:
json_filename = args.data_cache / '13_model_comparison.json'
results = fovea.pd.read_json(json_filename)
fig, axes = fovea.plot_model_comparison(results, fovea.all_model_names, fovea.all_datasets, do_masks=[True, False], figures_folder=args.figures_folder)


In [ ]:
json_filename = args.data_cache / '14_model_cn_comparison.json'
results = fovea.pd.read_json(json_filename)

fig, axes = fovea.plot_model_comparison(results, fovea.all_cn_model_names, fovea.all_datasets, do_masks=[True, False], figures_folder=args.figures_folder, save_name='14_model_convnext', exts=['svg'])

json_filename = args.data_cache / '13_model_comparison.json'
results = fovea.pd.read_json(json_filename)
fig, axes = fovea.plot_model_comparison(results, fovea.all_model_names, fovea.all_datasets, do_masks=[True, False], 
                                        fig=fig, axes=axes, figures_folder=args.figures_folder)


In [ ]:
combined_all_datasets = pd.concat([results13, results14], ignore_index=True)
combined_model_names = fovea.all_model_names.extend(fovea.all_cn_model_names)
combined_all_datasets = fovea.all_datasets.extend(fovea.all_datasets)

fig, axes = fovea.plot_model_comparison(combined_all_datasets, combined_model_names, combined_all_datasets, do_masks=[True, False], 
                                        figures_folder=args.figures_folder)
